In [1]:
import requests
import numpy as np
import pandas as pd
import yaml
from kaggle import Kaggle
import csv

In [2]:
def load_settings():
    with open('config.yaml', 'r') as sf:
        settings = yaml.load(sf.read())
    return settings

In [3]:
# Download dataset process
DATA_URL = "https://www.kaggle.com/crawford/weekly-sales-transactions/downloads/Sales_Transactions_Dataset_Weekly.csv"
SETTINGS = load_settings()

In [4]:
def chunk(df, n):
    for i in range(0, df.shape[0], n):
        yield df[i:i + n]

def cleanup(classA, arr):
    '''Cleanup Kaggle dataset since there are additional columns'''
    for i, x in enumerate(arr[0]):
        if (x.find('Normalized') == 0):
            continue
        if (x.find('MAX') == 0):
            continue
        if (x.find('MIN') == 0):
            continue
        classA.INDEX = i
    
    return [x[:classA.INDEX+1] for x in arr if len(x) > 1]

In [5]:
kaggle = Kaggle(SETTINGS)
response = kaggle.get_data(DATA_URL)

Login to Kaggle
Getting dataset


In [6]:
kaggle_data = kaggle.to_array(response.content.splitlines(), quoting=csv.QUOTE_NONE)

Transform data from text to array


In [7]:
processed_data = cleanup(kaggle, kaggle_data)
np_transpose_data = np.transpose(np.array(processed_data)) #transpose data so that product become the columns

In [8]:
# Convert to dataframme
fullData = pd.DataFrame(data=np_transpose_data[1:, 1:].astype(np.float), index=np_transpose_data[1:,0], columns=np_transpose_data[0,1:])

In [9]:
# Median for all product
median_data = fullData.median()

# Print sampling
print(median_data[:5])

P1    10.0
P2     3.5
P3     8.0
P4     8.0
P5     8.0
dtype: float64


In [10]:
# Mean for all product
mean_data = fullData.mean()

# Print sampling
print(mean_data[:5])

P1    9.634615
P2    3.980769
P3    8.692308
P4    8.269231
P5    8.461538
dtype: float64


In [11]:
# Min for all product
min_data = fullData.min()

# Print sampling
print(min_data[:5])

P1    3.0
P2    0.0
P3    3.0
P4    2.0
P5    3.0
dtype: float64


In [12]:
# Max for all product
max_data = fullData.max()

# Print sampling
print(max_data[:5])

P1    21.0
P2    10.0
P3    14.0
P4    19.0
P5    18.0
dtype: float64


In [13]:
# Best product based on volume
sum_of_product = fullData.sum()
index_array, = np.where(sum_of_product == sum_of_product.max())
best_product = (fullData.columns[index_array[0]], sum_of_product.max())
print(best_product)

('P409', 2220.0)


In [14]:
# Most promising product (ermerging product) quite vague
split_weekly = chunk(fullData, 1)
top_20_every_week = [x.sum().nlargest(20) for x in split_weekly]
top_20_last_3_week = top_20_every_week[-3:]

In [15]:
joined_group = pd.concat(top_20_last_3_week)
resetted = joined_group.reset_index()
resetted['count'] = 1
most_promising_product = resetted.groupby('index').agg({0:'sum', 'count': 'count'}).rename(columns={0:'sum'}).nlargest(1, 'count')
print(most_promising_product)

         sum  count
index              
P262   138.0      3


In [16]:
# 5 worst performing product on a biweekly basis
grouped = chunk(fullData, 2)
sum_grouped = [i.sum() for i in grouped]
worst_5_product_biweekly = [('Biweekly {}'.format(i+1), x.nsmallest(5)) for i, x in enumerate(sum_grouped)]

# Print sampling
print(worst_5_product_biweekly[:3])

[('Biweekly 1', P212    0.0
P213    0.0
P214    0.0
P215    0.0
P216    0.0
dtype: float64), ('Biweekly 2', P212    0.0
P213    0.0
P214    0.0
P215    0.0
P216    0.0
dtype: float64), ('Biweekly 3', P215    0.0
P216    0.0
P217    0.0
P218    0.0
P219    0.0
dtype: float64)]


In [17]:
# Get outliers for every product
outliers = fullData[~(np.abs(fullData - fullData.mean())<=(3*fullData.std()))]
print(outliers)

       P1  P2  P3    P4  P5  P6  P7  P8  P9  P10  ...   P810  P811  P812  \
W0    NaN NaN NaN   NaN NaN NaN NaN NaN NaN  NaN  ...    NaN   NaN   NaN   
W1    NaN NaN NaN   NaN NaN NaN NaN NaN NaN  NaN  ...    NaN   NaN   NaN   
W2    NaN NaN NaN   NaN NaN NaN NaN NaN NaN  NaN  ...    NaN   NaN   NaN   
W3    NaN NaN NaN   NaN NaN NaN NaN NaN NaN  NaN  ...    NaN   NaN   NaN   
W4    NaN NaN NaN   NaN NaN NaN NaN NaN NaN  NaN  ...    NaN   NaN   NaN   
W5    NaN NaN NaN   NaN NaN NaN NaN NaN NaN  NaN  ...    NaN   NaN   NaN   
W6    NaN NaN NaN   NaN NaN NaN NaN NaN NaN  NaN  ...    NaN   NaN   NaN   
W7   21.0 NaN NaN   NaN NaN NaN NaN NaN NaN  NaN  ...    NaN   NaN   NaN   
W8    NaN NaN NaN   NaN NaN NaN NaN NaN NaN  NaN  ...    NaN   NaN   NaN   
W9    NaN NaN NaN   NaN NaN NaN NaN NaN NaN  NaN  ...    NaN   NaN   NaN   
W10   NaN NaN NaN   NaN NaN NaN NaN NaN NaN  NaN  ...    NaN   NaN   NaN   
W11   NaN NaN NaN   NaN NaN NaN NaN NaN NaN  NaN  ...    NaN   NaN   6.0   
W12   NaN Na